# New method of using RAG to enahnce LLM knowledge for picking book

In [ ]:
import os
import requests
import pandas as pd
from langchain_community.llms import Ollama # newer version of this import from langchain_community.chat_models import ChatOllama idk if it changes anything though
from langchain_core.prompts import ChatPromptTemplate # Prompt templates convert raw user input to better input to the LLM (guide reponse)
from langchain_core.output_parsers import StrOutputParser # original model output is a message but this function parses it to a string (easier to work with)
from langchain_core.messages import HumanMessage, AIMessage # Used to frame history to LLM and retriever
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
file_path = r"goodreads_filtered.csv"

loader = CSVLoader(file_path=file_path,encoding='utf-8')
data = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
texts = text_splitter.split_documents(data) # Idk the difference between texts and data 

In [ ]:
texts1k = texts[0:1000]

ollama_emb = OllamaEmbeddings(  # 1k items took 38 mins, so 10 takes 6.3 hours
    model="llama3",
)
db = FAISS.from_documents(texts, ollama_emb)

# Saving FAISS vector db locally
#db.save_local(r"Vector DB Storage\FAISS_GoodreadsData1k")

In [ ]:
# Saving FAISS vector db locally
db.save_local(r"Vector DB Storage\FAISS_GoodreadsData10k")

In [ ]:
# Loading in the FAISS vector db from storage
ollama_emb = OllamaEmbeddings(
    model="llama3",
)
db = FAISS.load_local(r"Vector DB Storage\FAISS_GoodreadsData10k", ollama_emb, allow_dangerous_deserialization=True)

print(db.index.ntotal) # works with FAISS

In [ ]:
retriever = db.as_retriever()

In [ ]:
answer = db.similarity_search("horror and fantasy books for older patron", n=5)
for detail in answer:
    print(detail.page_content.split('\n')[0])
    print(detail.page_content.split('\n')[3])
    print('\n')

In [ ]:
answer[3]

### Instead of using an LLM to generate a list of book recs, I have have an LLM parse the user input extract and generate search terms that would be helpful with a book seach
### Them use those terms provided by the LLM into a pattern finding/search python function to generate a list of books that way (Broader context then we have all the info right there without having to do a check), kinda like how perplexity works where the LLM refines the search inquiry to optimize it  
### extract and generate key words from a query (LLM), database search using those generated key words to find books (python function), give book title and description to LLM to explain why these books are good in relation to original prompt. 

In [ ]:
from langchain_community.llms import Ollama # newer version of this import from langchain_community.chat_models import ChatOllama idk if it changes anything though
from langchain_core.prompts import ChatPromptTemplate # Prompt templates convert raw user input to better input to the LLM (guide reponse)
from langchain_core.output_parsers import StrOutputParser # original model output is a message but this function parses it to a string (easier to work with)
from langchain_core.messages import HumanMessage, AIMessage # Used to frame history to LLM and retriever
import pandas as pd
import time
import requests
import os
from filelock import FileLock, Timeout
import json

In [ ]:
# LLM that provides a list of book reccomendations (title only) in the format of a python list

model = "llama3" # mistral, llama2, kdl_copilot_llama3, llama3, llama2:13b
llm = Ollama(model=model, format='json') # how to structure LLM output: https://python.langchain.com/v0.2/docs/how_to/structured_output/

system = """Your are a helpful librarian AI assistant. Given a user query about reccomendations for books to read you parse that sentence \
    returning and gerating key words to be used in a pattern seach for relavent books in a book database. \
    The key words can be a combinations of words the user query as well as relavent words you think will be helpful search terms find relavent books for them. \
    Output your response in json. \
    Include at least 8 relevant key words. \ 

Here are some examples of proper json output based on an example prompts:

example_user: I like mystery book, what are some recommendations like that if I just read Louise Penny?
example_assistant: {{"keyword_list": [
    "Psychological depth",
    "Mystery",
    "Crime",
    "Suspense",
    "Character-driven"
    ]}}

example_user: What books should I read after 1984 by George Orwell??
example_assistant: {{"keyword_list": [
    "Dystopia",
    "Totalitarianism",
    "Big Brother",
    "Surveillance",
    "Political satire",
    "Orwellian"]}}
"""

prompt = ChatPromptTemplate.from_messages([  # LLM unreliable for ISBN number but does good for book titles
    ("system", system),
    ("user", "{input}")
])

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

orig_prompt = "I liked the mistborn series by sanderson, what are some books like it?"

output1 = chain.invoke({"input": orig_prompt}) 

print(output1) 

In [ ]:
import json
#LLm outputs info as a string so we need to load it as json to be able index into it 
llm_output = json.loads(output1)

listers = llm_output["keyword_list"] 

##### Pattern matching exploring

In [ ]:
# Pattern matching search which I will not do 
import pandas as pd

# Load the data from CSV file
df = pd.read_excel(r'Complete_Monster_List.xlsx')

# List of search terms
search_terms = ['Science fiction', 'Juvenile', 'Fantasy fiction', 'Magic']

# Function to perform pattern search
def search_pattern(row, terms):
    genre = row['Specific Genre'] if pd.notna(row['Specific Genre']) else ''
    audience = row['Audiance'] if pd.notna(row['Audiance']) else ''
    subject = row['Subject'] if pd.notna(row['Subject']) else ''
    return any(term in genre or term in audience or term in subject for term in terms)

# Apply function to DataFrame
df['match'] = df.apply(search_pattern, terms=listers, axis=1)

# Filter DataFrame based on matches
matched_df = df[df['match']]

# Display the result
print(matched_df)

In [ ]:
books[books['Author'].str.contains("Rebecca", case=False, na=False)] # na is used to ignore missing values


filtered_books = books[books['Author'].str.contains("Rebecca", case=False, na=False)]
selected_columns = filtered_books[['Title', 'Author', 'Item Type', 'Rating']] # selecting only specific columns to be displayed in the output df

In [ ]:
for column in ['Title', 'Author', 'Item Type', 'Rating']:
    # Check if any keyword matches the value in the current column using str.contains()
    for keyword in keywords:
        if books[books[column].str.contains("Rebecca", case=False, na=False)]:
            # If a match is found, add 1 to the ranking score
            ranking_score += 1

ranking_scores.append(ranking_score)

In [ ]:
for index, row in books.iterrows():
    print(row['Title'], row['Author'], row['Subject'])

In [ ]:
def keyword_search(dataframe, keywords):
    # Initialize an empty list to store the ranking scores
    ranking_scores = []

    # Iterate through each index and row in the DataFrame
    for index, row in enumerate(dataframe):
        # Initialize ranking score for the current row
        ranking_score = 0
        
        # Iterate through each column of interest
        for column in ['Title', 'Author', 'Item Type', 'Rating']:
            # Check if any keyword matches the value in the current column using str.contains()
            for keyword in keywords:
                if row[column].str.contains(keyword, case=False, na=False).any():
                    # If a match is found, increment the ranking score
                    ranking_score += 1
        
        # Append the ranking score for the current row to the list
        ranking_scores.append(ranking_score)
    
    # Add ranking_scores list as a new column to the DataFrame
    dataframe['Ranking Score'] = ranking_scores
    
    # Sort the DataFrame based on the ranking score in descending order
    sorted_df = dataframe.sort_values(by='Ranking Score', ascending=False)
    
    # Reset index for better presentation
    sorted_df.reset_index(drop=True, inplace=True)
    
    # Drop the ranking score column before returning the DataFrame
    sorted_df.drop(columns=['Ranking Score'], inplace=True)
    
    return sorted_df

In [ ]:
# Assuming you have a DataFrame called 'books'

# List of keywords to search for
keywords = ['fantasy', 'adventure']

# Perform keyword search
search_result = keyword_search(books, keywords)

# Display the search result
print(search_result)

# 3rd method! Normal rec into LLM explaination

## Collaborative filtering ex

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def create_user_item_matrix(data):
    return pd.DataFrame(data).T

def print_matrix(user_item_matrix):
    print("\nUser-Item Rating Matrix:")
    print("------------------------")
    
    max_user_length = max(len(user) for user in user_item_matrix.index)
    max_book_length = max(len(book) for book in user_item_matrix.columns)
    
    print(f"{' ':<{max_user_length+2}}", end="")
    for book in user_item_matrix.columns:
        print(f"{book:<{max_book_length+2}}", end="")
    print()
    
    for user in user_item_matrix.index:
        print(f"{user:<{max_user_length+2}}", end="")
        for rating in user_item_matrix.loc[user]:
            if rating == 0:
                print(f"{'—':<{max_book_length+2}}", end="")
            else:
                print(f"{rating:<{max_book_length+2}}", end="")
        print()
    
    total_cells = user_item_matrix.size
    filled_cells = (user_item_matrix != 0).sum().sum()
    sparsity = 1 - (filled_cells / total_cells)
    print(f"\nMatrix Sparsity: {sparsity:.2%}")
    print(f"Filled Cells: {filled_cells}/{total_cells}")

def get_recommendations(user_item_matrix, user_id, n=5):
    user_similarity = cosine_similarity(user_item_matrix)
    similar_users = user_similarity[user_id].argsort()[::-1][1:]
    user_ratings = user_item_matrix.iloc[user_id]
    unrated_books = user_ratings[user_ratings == 0].index
    
    recommendations = {}
    for book in unrated_books:
        pred_rating = 0
        total_similarity = 0
        for similar_user in similar_users:
            if user_item_matrix.iloc[similar_user][book] != 0:
                similarity = user_similarity[user_id][similar_user]
                rating = user_item_matrix.iloc[similar_user][book]
                pred_rating += similarity * rating
                total_similarity += similarity
        if total_similarity > 0:
            recommendations[book] = pred_rating / total_similarity
    
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return sorted_recommendations[:n]

# Sample dataset
data = {
    'User1': {'Book1': 4, 'Book2': 3, 'Book3': 5, 'Book4': 0, 'Book5': 0},
    'User2': {'Book1': 5, 'Book2': 0, 'Book3': 4, 'Book4': 2, 'Book5': 1},
    'User3': {'Book1': 3, 'Book2': 4, 'Book3': 0, 'Book4': 5, 'Book5': 2},
    'User4': {'Book1': 0, 'Book2': 5, 'Book3': 3, 'Book4': 4, 'Book5': 0},
    'User5': {'Book1': 2, 'Book2': 0, 'Book3': 0, 'Book4': 3, 'Book5': 5}
}

# Create the user-item matrix
user_item_matrix = create_user_item_matrix(data)

# Print the matrix
print_matrix(user_item_matrix)

# Get recommendations for User1
user_id = 4 # User1's index
recommendations = get_recommendations(user_item_matrix, user_id)

print("\nRecommendations for User1:")
for book, rating in recommendations:
    print(f"{book}: Predicted rating = {rating:.2f}")

# NEW NEW idea! 

#### Bibliocommons has book read alikes (from novelist) for every book in our collection based on URL format: "https://kdl.bibliocommons.com/v2/record/S174C605581/recommendations" only adding 'recommendations' after each item. 
#### If we could gather a dict of all kdl items and their associated bibliocommons id, we could scrape those read a likes for an LLM to synthesize 